In [1]:
import pandas as pd

In [25]:
# df = pd.read_csv('https://raw.githubusercontent.com/Alois-Software/data-modeling/word-embedding/social_media_remove_long_posts.csv', header=0)

In [2]:
df = pd.read_csv('social_media_remove_long_posts.csv', header=0)

In [3]:
df

,text
0,Alzheimers Dementia and Support ‘ADS’ is one o...
1,"Hello Everyone,\nIt is that time of year again..."
2,Slough Borough Council is seeking the views of...
3,Not heard from us in a while?\n\nWe are not ig...
4,Less than 3months to go...\n\nGet yourself rea...
...,...
508,I've been doing a lot of thinking about what i...
509,"I needed this today, on Father’s Day."
510,Mom just told me she don't feel good. She tell...
511,Wishing you a rainbow of happiness and good ch...


In [4]:
df['text']

0      Alzheimers Dementia and Support ‘ADS’ is one o...
1      Hello Everyone,\nIt is that time of year again...
2      Slough Borough Council is seeking the views of...
3      Not heard from us in a while?\n\nWe are not ig...
4      Less than 3months to go...\n\nGet yourself rea...
                             ...                        
508    I've been doing a lot of thinking about what i...
509                I needed this today, on Father’s Day.
510    Mom just told me she don't feel good. She tell...
511    Wishing you a rainbow of happiness and good ch...
512    The Red Arrows flying over Maidenhead after th...
Name: text, Length: 513, dtype: object

In [5]:
!pip install transformers

In [6]:
!pip install torch

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [8]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
batch_1 = df['text']

In [10]:
tokenized = batch_1.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
np.array(padded).shape

(513, 473)

In [13]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(513, 473)

In [14]:
max_len

473

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)